In [1]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.util import ngrams
from collections import defaultdict, Counter
import re

nltk.download("wordnet")
nltk.download("punkt")
nltk.download("punkt_tab")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\saisab31\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\saisab31\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\saisab31\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
# Updated documents for Query Operations Lab
documents = [
    "I loved the movie, the plot was thrilling and the characters were amazing.",
    "The film was boring and predictable, not worth my time.",
    "An enjoyable movie with some funny moments and great acting.",
    "Terrible direction, bad script, and poor editing ruined the movie.",
]

# Example query
query = "thrilling movie"

# Vocabulary
vocab = ["movie", "plot", "thrilling", "characters", "boring", "predictable",
         "enjoyable", "funny", "acting", "direction", "script", "editing", "ruined"]

# Corpus
corpus = " ".join(documents)



---

## 1. Query Expansion

**Query expansion** improves search by adding **semantically related terms** to the original query.  

- Uses **WordNet** to retrieve **synonyms (lemmas)** for query words.
- Helps retrieve documents containing related words rather than exact matches.

**Example**:

```python
query_expansion_wordnet("thrilling")


In [3]:
def query_expansion_wordnet(query):
    words = nltk.word_tokenize(query)
    expanded_query = set(words)
    for word in words:
        for syn in wn.synsets(word):
            for lemma in syn.lemmas():
                expanded_query.add(lemma.name().replace("_", " "))
    return list(expanded_query)


print("1. Query Expansion (WordNet):")
print(query_expansion_wordnet(query))

1. Query Expansion (WordNet):
['vibrate', 'pic', 'movie', 'exalt', 'flick', 'beatify', 'inebriate', 'electrifying', 'moving-picture show', 'tickle', 'picture', 'motion-picture show', 'exhilarate', 'throb', 'tickle pink', 'thrilling', 'shudder', 'moving picture', 'shiver', 'film', 'thrill', 'picture show', 'motion picture']


---
### 2A. Spelling Correction: Edit Distance (Levenshtein Distance)

This section demonstrates **spelling correction using Edit Distance**. The code consists of two functions:

#### 1. `edit_distance(w1, w2)`

- Computes the **minimum number of single-character edits** (insertions, deletions, substitutions) to convert `w1` to `w2`.
- Uses a **dynamic programming matrix** `dp` of size `(len(w1)+1) x (len(w2)+1)`.
- Filling the matrix:
  - `dp[i][0] = i` → deleting all characters from `w1`
  - `dp[0][j] = j` → inserting all characters from `w2`
  - `dp[i][j] = dp[i-1][j-1]` if characters match
  - `dp[i][j] = 1 + min(deletion, insertion, substitution)` if characters differ

#### 2. `correct_by_edit_distance(word, vocab)`

- Finds the **closest word** in the vocabulary to the misspelled word.
- Computes `edit_distance` for each vocabulary word.
- Returns the word with **minimum distance** as the correction.

---

In [5]:
# A. Edit Distance
def edit_distance(w1, w2):
    dp = [[0] * (len(w2) + 1) for _ in range(len(w1) + 1)]
    for i in range(len(w1) + 1):
        for j in range(len(w2) + 1):
            if i == 0:
                dp[i][j] = j
            elif j == 0:
                dp[i][j] = i
            elif w1[i - 1] == w2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1])
    return dp[-1][-1]


def correct_by_edit_distance(word, vocab):
    min_dist = float("inf")
    correction = word
    for w in vocab:
        dist = edit_distance(word, w)
        if dist < min_dist:
            min_dist = dist
            correction = w
    return correction


print("\n2A. Edit Distance Correction:")
print(correct_by_edit_distance("thrillnig", vocab))


2A. Edit Distance Correction:
thrilling


In [7]:
# B. K-Gram Index
def generate_k_grams(word, k=3):
    word = f"${word}$"
    return [word[i : i + k] for i in range(len(word) - k + 1)]


def kgram_index(vocab, k=3):
    index = defaultdict(set)
    for word in vocab:
        grams = generate_k_grams(word, k)
        for g in grams:
            index[g].add(word)
    return index


def correct_by_kgram(word, index, vocab, k=3):
    grams = generate_k_grams(word, k)
    candidates = Counter()
    for g in grams:
        for cand in index.get(g, []):
            candidates[cand] += 1
    if not candidates:
        return word
    # Get candidates with highest overlap
    max_overlap = candidates.most_common(1)[0][1]
    best_candidates = [w for w, c in candidates.items() if c == max_overlap]
    # Choose the one with smallest edit distance
    return min(best_candidates, key=lambda w: edit_distance(word, w))


print("\n2B. K-Gram Correction:")
k_index = kgram_index(vocab)
print(correct_by_kgram("scrit", k_index, vocab))


2B. K-Gram Correction:
script


In [12]:
# C. Context Sensitive Correction (Bigram Based)
def train_bigram_model(corpus):
    tokens = nltk.word_tokenize(corpus.lower())
    bigrams = list(ngrams(tokens, 2))
    model = Counter(bigrams)
    return model


def correct_contextually(word_list, model, vocab):
    corrected = [word_list[0]]
    for i in range(1, len(word_list)):
        prev_word = corrected[-1]
        word = word_list[i]
        candidates = [word, correct_by_edit_distance(word, vocab)]
        best_word = max(candidates, key=lambda w: model.get((prev_word, w), 0))
        corrected.append(best_word)
    return corrected


print("\n2C. Context Sensitive:")
bigram_model = train_bigram_model(corpus)
print(correct_contextually(["boring", "bor"], bigram_model, vocab))


2C. Context Sensitive:
['boring', 'bor']


# 3. Query Language Interpreter
A. Single-Word Queries

Matches documents containing the exact word.
Example:

movie → [
    "I loved the movie, the plot was thrilling and the characters were amazing.",
    "An enjoyable movie with some funny moments and great acting.",
    "Terrible direction, bad script, and poor editing ruined the movie."
]

B. Boolean Queries

Combines terms using logical operators: AND, OR, NOT.
Example:

thrilling AND movie → [
    "I loved the movie, the plot was thrilling and the characters were amazing."
]

C. Natural Language Queries

Interprets queries in plain English by tokenizing into keywords.
Example:

"Was the movie exciting?" → [
    "I loved the movie, the plot was thrilling and the characters were amazing.",
    "An enjoyable movie with some funny moments and great acting."
]

D. Structural Queries

Targets specific fields (title, body, etc.).

Example: title:script returns documents containing "script".

In [14]:
def single_word_query(word, documents):
    return [doc for doc in documents if word in doc.lower()]


print("\n3A. Single-word Query:")
print(single_word_query("plot", documents))


3A. Single-word Query:
['I loved the movie, the plot was thrilling and the characters were amazing.']


In [15]:
def boolean_query(q, documents):
    terms = q.lower().split()
    result = set(documents)
    if "and" in terms:
        terms = [t for t in terms if t != "and"]
        result = [doc for doc in documents if all(t in doc.lower() for t in terms)]
    elif "or" in terms:
        terms = [t for t in terms if t != "or"]
        result = [doc for doc in documents if any(t in doc.lower() for t in terms)]
    elif "not" in terms:
        idx = terms.index("not")
        term = terms[idx + 1]
        result = [doc for doc in documents if term not in doc.lower()]
    return result


print("\n3B. Boolean Query:")
print(boolean_query("movie AND plot", documents))


3B. Boolean Query:
['I loved the movie, the plot was thrilling and the characters were amazing.']


In [16]:
def natural_language_query(nl_query, documents):
    tokens = nltk.word_tokenize(nl_query.lower())
    return [doc for doc in documents if any(t in doc.lower() for t in tokens)]


print("\n3C. Natural Language Query:")
print(natural_language_query("Was the plot good?", documents))


3C. Natural Language Query:
['I loved the movie, the plot was thrilling and the characters were amazing.', 'The film was boring and predictable, not worth my time.', 'Terrible direction, bad script, and poor editing ruined the movie.']


In [17]:
def structural_query(structure_query, documents):
    # Dummy: match title:xxx or body:xxx
    m = re.match(r"(title|body):(\w+)", structure_query.lower())
    if m:
        field, word = m.groups()
        return [doc for doc in documents if word in doc.lower()]
    return []


print("\n3D. Structural Query:")
print(structural_query("title:movie", documents))


3D. Structural Query:
['I loved the movie, the plot was thrilling and the characters were amazing.', 'An enjoyable movie with some funny moments and great acting.', 'Terrible direction, bad script, and poor editing ruined the movie.']
